In [1]:
!pip install selenium

     |████████████████████████████████| 904 kB 5.4 MB/s eta 0:00:01


In [65]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://store.steampowered.com/search/?filter=topsellers'

#user_agent = {'User-agent': 'Mozilla/5.0'}
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [47]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [48]:
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]

In [49]:
len(rows)

50

In [50]:
#this was sufficient to scroll to the bottom of the top sellers page for steam, pulling code from scraping list
for i in range(500):
    #Scroll
    driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
    )
    time.sleep(1)

In [53]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [54]:
#successfully scraped 16.5k elements, hopefully all games.
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]
rows[0]
len(rows)

16700

In [55]:
#pulling code created in 'scraping_list' notebook to create dataframe.
#dataframe index is game name, 

games = {}

for row in rows:
    title = (row.find('span', class_='title').text)
    url = row.get('href')
    release_date = (row.find('div', class_='col search_released responsive_secondrow').text)
    if (row.find('div', class_='col search_discount responsive_secondrow').text.strip()) == '':
        been_sale = 0
    else:
        been_sale = 1
    if been_sale == 0:
        orig_price = (row.find('div', class_='col search_price responsive_secondrow').text.strip())
    else:
        orig_price = (row.find('strike').text)
    games[title] = (url, 
                    release_date,
                    been_sale,
                    orig_price)
    

In [56]:
gamesdf1 = pd.DataFrame(games).T
gamesdf1.columns = ['link', 'release_date', 'been_sale', 'orig_price']

gamesdf1.shape

(16560, 4)

In [57]:
#no nulls!!
gamesdf1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16560 entries, OUTRIDERS to Dinosaur Hunt - Stegosaurus Expansion Pack
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   link          16560 non-null  object
 1   release_date  16560 non-null  object
 2   been_sale     16560 non-null  object
 3   orig_price    16560 non-null  object
dtypes: object(4)
memory usage: 646.9+ KB


In [58]:
gamesdf1.head()

,link,release_date,been_sale,orig_price
OUTRIDERS,https://store.steampowered.com/sub/504022/?snr...,"Apr 1, 2021",0,$59.99
Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,"Jun 3, 2020",1,$39.99
Halo: The Master Chief Collection,https://store.steampowered.com/sub/376282/?snr...,"Dec 3, 2019",1,$39.99
It Takes Two,https://store.steampowered.com/app/1426210/It_...,"Mar 26, 2021",0,$39.99
Valheim,https://store.steampowered.com/app/892970/Valh...,"Feb 2, 2021",0,$19.99


In [64]:
gamesdf1.tail()

,link,release_date,been_sale,orig_price
Dinosaur Hunt - Dragon Hunter Expansion Pack,https://store.steampowered.com/app/408102/Dino...,"Dec 18, 2015",0,$0.99
Dinosaur Hunt - Giant Spiders Hunter Expansion Pack,https://store.steampowered.com/app/431191/Dino...,"Mar 16, 2016",0,$0.99
Dinosaur Hunt - Brontosaurus Expansion Pack,https://store.steampowered.com/app/434942/Dino...,"Mar 16, 2016",0,$0.99
"Dinosaur Hunt - Vampires, Gargoyles, Mutants Hunter Expansion Pack",https://store.steampowered.com/app/431190/Dino...,"Jan 22, 2016",0,$0.99
Dinosaur Hunt - Stegosaurus Expansion Pack,https://store.steampowered.com/app/408100/Dino...,"Nov 13, 2015",0,$0.99


# attempt 1 to pull all game page data

Just "successfully" ran the function to pull all game data for the first 50 games. several nulls. going to run the code on here overnight, to see if there are enough non-null games out of the 16k to run a suitable model. if not, will have to write conditions for variance in page layouts, or supplement my scraped data with steamspy/steam's api

In [69]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(5):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [70]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))
    

AttributeError: 'NoneType' object has no attribute 'text'

In [77]:
games_info = pd.DataFrame(games_info)

In [73]:
gamesdf1.iloc[312]

link            https://store.steampowered.com/app/10090/Call_...
release_date                                         Nov 18, 2008
been_sale                                                       0
orig_price                                                 $19.99
Name: Call of Duty: World at War, dtype: object

In [ ]:
# first attempt failed, but only after 300 iterations. going to look into nulls now, but am going
# to rewrite the function to account for failing at EVERY find, not just first one

In [78]:
import numpy as np
games_info.replace('HELP', np.nan, inplace)

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,NaN,NaN,NaN,NaN,NaN,NaN,"[Action, Adventure, RPG, People Can Fly]",4.0,NaN
1,Sea of Thieves,"109,901",Very Positive,13,"[Adventure, Multiplayer, Open World, Pirates, ...",20.0,"[Action, Adventure]",2.0,Xbox Game Studios
2,NaN,NaN,NaN,NaN,NaN,NaN,"[Action, 343 Industries, Splash Damage, Ruffia...",8.0,NaN
3,It Takes Two,"10,408",Overwhelmingly Positive,12,"[Co-op, Adventure, 3D Platformer, Puzzle, Puzz...",20.0,"[Action, Adventure, Hazelight]",3.0,Electronic Arts
4,Valheim,"215,050",Overwhelmingly Positive,12,"[Open World Survival Craft, Online Co-Op, Open...",20.0,"[Action, Adventure, Indie, RPG, Early Access, ...",6.0,Coffee Stain Publishing
...,...,...,...,...,...,...,...,...,...
306,RimWorld - Royalty,NaN,Very Positive,NaN,"[Indie, Strategy, Simulation, Sandbox, Base Bu...",12.0,"[Indie, Simulation]",2.0,Ludeon Studios
307,Pacify,"21,164",Very Positive,16,"[Horror, Multiplayer, Co-op, Survival Horror, ...",20.0,"[Action, Adventure, Casual, Indie, Strategy]",5.0,Hitchcock Games
308,NaN,NaN,NaN,NaN,NaN,NaN,"[Action, RPG, Violent, OVERKILL - a Starbreeze...",5.0,NaN
309,Eco,"7,060",Very Positive,NaN,"[Open World Survival Craft, Survival, Multipla...",20.0,"[Indie, Simulation, Early Access]",3.0,Strange Loop Games


In [79]:
games_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          311 non-null    object
 1   num_reviews    311 non-null    object
 2   review_rating  311 non-null    object
 3   num_languages  311 non-null    object
 4   tags           311 non-null    object
 5   num_tags       311 non-null    object
 6   genres         311 non-null    object
 7   num_genres     311 non-null    object
 8   publisher      311 non-null    object
dtypes: object(9)
memory usage: 22.0+ KB
